In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

import lightgbm as lgb
#import xgboost as xgb
#import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'target'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 20000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
# Load the training data
train = pd.read_csv("../input/30-days-of-ml/train.csv")
test = pd.read_csv("../input/30-days-of-ml/test.csv")

# Preview the data
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [5]:
# Low MI scores
for pp in ["cat2","cat3","cat4","cat6"]:
    train.pop(pp)
    test.pop(pp)
train.head()

,id,cat0,cat1,cat5,cat7,cat8,cat9,cont0,cont1,cont2,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,E,C,N,0.201470,-0.014822,0.669699,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,D,F,A,O,0.743068,0.367411,1.021605,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,D,D,A,F,0.742708,0.310383,-0.012673,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,D,E,C,K,0.429551,0.620998,0.577942,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,D,E,A,N,1.058291,0.367492,-0.052389,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [6]:
# Separate target from features
y = train['target']
features = train.drop(['id','target'], axis=1)

# Preview features
features.head()

,cat0,cat1,cat5,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,B,B,B,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,B,B,D,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,A,A,D,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,B,B,D,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,A,A,D,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


# Preprocessing

In [7]:

# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# ordinal-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()
ordinal_encoder = OrdinalEncoder()
X[object_cols] = ordinal_encoder.fit_transform(features[object_cols])
X_test[object_cols] = ordinal_encoder.transform(test[object_cols])

# Preview the ordinal-encoded features
X.head()


,cat0,cat1,cat5,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,1.0,1.0,1.0,4.0,2.0,13.0,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,1.0,1.0,3.0,5.0,0.0,14.0,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,0.0,0.0,3.0,3.0,0.0,5.0,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,1.0,1.0,3.0,4.0,2.0,10.0,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,0.0,0.0,3.0,4.0,0.0,13.0,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


In [8]:
'''
# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# one-hot-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()

oh_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
X_ohe = oh_encoder.fit_transform(features[object_cols])
X_test_ohe = oh_encoder.transform(test[object_cols])

X_ohe = pd.DataFrame(X_ohe, columns=[f"ohe_{i}" for i in range(X_ohe.shape[1])])
X_test_ohe = pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])

X = pd.concat([X, X_ohe], axis=1)
X_test = pd.concat([X_test, X_test_ohe], axis=1)
X = X.drop(object_cols, axis=1)
X_test = X_test.drop(object_cols, axis=1)
    
# Preview the one-hot-encoded features
X.head()
'''

'\n# List of categorical columns\nobject_cols = [col for col in features.columns if \'cat\' in col]\n\n# one-hot-encode categorical columns\nX = features.copy()\nX_test = test.drop([\'id\'], axis=1).copy()\n\noh_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")\nX_ohe = oh_encoder.fit_transform(features[object_cols])\nX_test_ohe = oh_encoder.transform(test[object_cols])\n\nX_ohe = pd.DataFrame(X_ohe, columns=[f"ohe_{i}" for i in range(X_ohe.shape[1])])\nX_test_ohe = pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])\n\nX = pd.concat([X, X_ohe], axis=1)\nX_test = pd.concat([X_test, X_test_ohe], axis=1)\nX = X.drop(object_cols, axis=1)\nX_test = X_test.drop(object_cols, axis=1)\n    \n# Preview the one-hot-encoded features\nX.head()\n'

In [9]:
scale_features = [col for col in features.columns if 'cont' in col]

ss = StandardScaler()
X[scale_features] = ss.fit_transform(features[scale_features])
X_test[scale_features] = ss.transform(test[scale_features])

In [10]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Optuna

In [11]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [12]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=X, y=y):
  """
  """
  param_space = {
               #'device':'gpu',  # Use GPU acceleration
               #'boosting_type': 'gbdt',
               'reg_lambda':trial.suggest_uniform('reg_lambda', 0.001, 25),
              'reg_alpha':trial.suggest_uniform('reg_alpha', 3, 27),
                'subsample': trial.suggest_uniform('subsample', 0.2, 0.6),
              'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.1, 0.4),
                #'subsample_freq': trial.suggest_int('subsample_freq', 1, 10),
               'learning_rate':trial.suggest_uniform('learning_rate', 2e-2, 6.95e-2),
                 'min_child_samples':trial.suggest_int('min_child_samples', 5, 55),
              'num_leaves':trial.suggest_int('num_leaves', 87, 187),
              'max_depth':trial.suggest_int('max_depth', 2, 15),
              #'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 0.005),
              #'class_weight':trial.suggest_categorical('class_weight',['balanced',None]),
               'n_estimators':N_ESTIMATORS,
                'n_jobs' : -1,
              'metric':'rmse',
              'max_bin':trial.suggest_int('max_bin', 494, 894),
              'cat_smooth':trial.suggest_int('cat_smooth', 37, 87),
              'cat_l2':trial.suggest_loguniform('cat_l2', 50, 98)
                }
            
  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  #lgb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  pred=np.zeros(y_va.shape[0])
    
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
    
  #X_tr = apply_noise_row(X_tr)

  
  #start = time.time()
  for inseed in seed_list:
    param_space['random_state'] = inseed

    model = lgb.LGBMRegressor(**param_space)
    model.fit(
        X_tr, 
        y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric='rmse',
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE,
        categorical_feature=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
        #callbacks=[optuna.integration.LightGBMPruningCallback(trial, metric='rmse')],
    )


    pred += model.predict(X_va) / len(seed_list)
    
  #elapsed = time.time() - start
  rmse = mean_squared_error(y_va, pred, squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [13]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-25 23:56:31,103] A new study created in memory with name: no-name-5dd489b2-a311-43f9-8f29-a049d842e4f1


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.724652
[2000]	valid_0's rmse: 0.721277
[3000]	valid_0's rmse: 0.719567
[4000]	valid_0's rmse: 0.718587
[5000]	valid_0's rmse: 0.717973
[6000]	valid_0's rmse: 0.717556
[7000]	valid_0's rmse: 0.717375
[8000]	valid_0's rmse: 0.71734
Early stopping, best iteration is:
[7800]	valid_0's rmse: 0.717319
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.724603
[2000]	valid_0's rmse: 0.721304
[3000]	valid_0's rmse: 0.71953
[4000]	valid_0's rmse: 0.718635
[5000]	valid_0's rmse: 0.718075
[6000]	valid_0's rmse: 0.717749
[7000]	valid_0's rmse: 0.717467
Early stopping, best iteration is:
[7394]	valid_0's rmse: 0.717408


[I 2021-08-25 23:59:19,541] Trial 0 finished with value: 0.7172494952683935 and parameters: {'reg_lambda': 12.448419725180374, 'reg_alpha': 4.982647038535252, 'subsample': 0.4310059949444464, 'colsample_bytree': 0.2579181572031725, 'learning_rate': 0.0421451971220899, 'min_child_samples': 28, 'num_leaves': 120, 'max_depth': 2, 'max_bin': 614, 'cat_smooth': 58, 'cat_l2': 86.82538385789324}. Best is trial 0 with value: 0.7172494952683935.


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.717737
Early stopping, best iteration is:
[1340]	valid_0's rmse: 0.717562
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.718173
Early stopping, best iteration is:
[1570]	valid_0's rmse: 0.717662


[I 2021-08-26 00:00:06,756] Trial 1 finished with value: 0.7171721640380457 and parameters: {'reg_lambda': 16.011130453847322, 'reg_alpha': 6.757455032499683, 'subsample': 0.24892822034485254, 'colsample_bytree': 0.10121420184848116, 'learning_rate': 0.06559381728938582, 'min_child_samples': 16, 'num_leaves': 167, 'max_depth': 5, 'max_bin': 741, 'cat_smooth': 66, 'cat_l2': 57.89927784408105}. Best is trial 1 with value: 0.7171721640380457.


...

Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.723774
[2000]	valid_0's rmse: 0.720145
[3000]	valid_0's rmse: 0.718868
[4000]	valid_0's rmse: 0.718239
[5000]	valid_0's rmse: 0.717862
[6000]	valid_0's rmse: 0.717623
[7000]	valid_0's rmse: 0.717509
[8000]	valid_0's rmse: 0.717411
[9000]	valid_0's rmse: 0.717355
[10000]	valid_0's rmse: 0.717315
Early stopping, best iteration is:
[9995]	valid_0's rmse: 0.717314
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.724834
[2000]	valid_0's rmse: 0.721543
[3000]	valid_0's rmse: 0.719574
[4000]	valid_0's rmse: 0.718691
[5000]	valid_0's rmse: 0.718139
[6000]	valid_0's rmse: 0.717795
[7000]	valid_0's rmse: 0.717586
[8000]	valid_0's rmse: 0.717438
[9000]	valid_0's rmse: 0.717393
[10000]	valid_0's rmse: 0.717336
Early stopping, best iteration is:
[10539]	valid_0's rmse: 0.717317


[I 2021-08-26 00:52:18,817] Trial 28 finished with value: 0.7170157307892239 and parameters: {'reg_lambda': 19.47607712035561, 'reg_alpha': 21.747681447737023, 'subsample': 0.4059186039123447, 'colsample_bytree': 0.10260318507191415, 'learning_rate': 0.03436568478792896, 'min_child_samples': 42, 'num_leaves': 173, 'max_depth': 3, 'max_bin': 781, 'cat_smooth': 43, 'cat_l2': 81.03366487614083}. Best is trial 26 with value: 0.716813599897907.


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.718477
[2000]	valid_0's rmse: 0.717899
Early stopping, best iteration is:
[1980]	valid_0's rmse: 0.71788
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.718405
[2000]	valid_0's rmse: 0.717935
Early stopping, best iteration is:
[1811]	valid_0's rmse: 0.71787


[I 2021-08-26 00:53:22,130] Trial 29 finished with value: 0.7176044457798756 and parameters: {'reg_lambda': 22.474692440539222, 'reg_alpha': 20.328307792257203, 'subsample': 0.448027917969831, 'colsample_bytree': 0.2813080746270542, 'learning_rate': 0.04145658909748892, 'min_child_samples': 29, 'num_leaves': 155, 'max_depth': 5, 'max_bin': 713, 'cat_smooth': 56, 'cat_l2': 87.98154242380733}. Best is trial 26 with value: 0.716813599897907.


Number of finished trials: 30
Best trial: {'reg_lambda': 22.64241900807383, 'reg_alpha': 22.32970088488026, 'subsample': 0.3237257183367251, 'colsample_bytree': 0.15268338858066302, 'learning_rate': 0.03289518963074013, 'min_child_samples': 34, 'num_leaves': 140, 'max_depth': 3, 'max_bin': 705, 'cat_smooth': 44, 'cat_l2': 82.37637091520239}


In [14]:
study.best_params

{'reg_lambda': 22.64241900807383,
 'reg_alpha': 22.32970088488026,
 'subsample': 0.3237257183367251,
 'colsample_bytree': 0.15268338858066302,
 'learning_rate': 0.03289518963074013,
 'min_child_samples': 34,
 'num_leaves': 140,
 'max_depth': 3,
 'max_bin': 705,
 'cat_smooth': 44,
 'cat_l2': 82.37637091520239}

# Log

====== Ordinal encoding =========

0.7169747685796982 no noise ver1

====== One-hot encoding =========

0.724412184059166 no noise ver2